Colab is making it easier than ever to integrate powerful Generative AI capabilities into your projects. We are launching public preview for a simple and intuitive Python library (google.colab.ai) to access state-of-the-art language models directly within Colab environments. All users have free access to most popular LLMs, while paid users have access to a wider selection of models. This means users can spend less time on configuration and set up and more time bringing their ideas to life. With just a few lines of code, you can now perform a variety of tasks:
- Generate text
- Translate languages
- Write creative content
- Categorize text

Happy Coding!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/Getting_started_with_google_colab_ai.ipynb)

In [1]:
# @title List available models
from google.colab import ai

ai.list_models()

['google/gemini-2.5-flash', 'google/gemini-2.5-flash-lite']

Choosing a Model
The model names give you a hint about their capabilities and intended use:

Pro: These are the most capable models, ideal for complex reasoning, creative tasks, and detailed analysis.

Flash: These models are optimized for high speed and efficiency, making them great for summarization, chat applications, and tasks requiring rapid responses.

Gemma: These are lightweight, open-weight models suitable for a variety of text generation tasks and are great for experimentation.

In [2]:
# @title Simple batch generation example
# Only text-to-text input/output is supported
from google.colab import ai

response = ai.generate_text("What is the capital of France?")
print(response)

The capital of France is **Paris**.


In [4]:
# @title Choose a different model
from google.colab import ai

response = ai.generate_text("What is the capital of England", model_name='google/gemini-2.5-flash-lite')
print(response)

The capital of England is **London**.


For longer text generations, you can stream the response. This displays the output token by token as it's generated, rather than waiting for the entire response to complete. This provides a more interactive and responsive experience. To enable this, simply set stream=True.

In [5]:
# @title Simple streaming example
from google.colab import ai

stream = ai.generate_text("Tell me a short story.", stream=True)
for text in stream:
  print(text, end='')

Silas lived in a quiet corner of the world, a place where the scent of sawdust and polished wood clung to the air like memory. He wasn't a man of grand words or bold adventures; he was a music box maker, his days spent coaxing fragile melodies from tiny gears and intricate mechanisms. His workshop, nestled above a sleepy cobblestone street, hummed with the soft clicks and whirs of nascent tunes.

One crisp autumn morning, a peculiar silence began to creep into town. It wasn't the absence of noise, but the absence of *lilt*. The baker's wife, usually humming a cheerful tune as she kneaded dough, found her song hollow. The street fiddler’s jigs lost their bounce, becoming flat and listless. Even the church choir, renowned for its soaring harmonies, began to sound strangely muted, as if a vital spark had been extinguished.

Silas noticed it first. He worked with melodies, after all. He’d wind a miniature carousel tune, and instead of its usual bright twinkle, it would sigh, a faint echo o

In [6]:
#@title Text formatting setup
#code is not necessary for colab.ai, but is useful in fomatting text chunks
import sys

class LineWrapper:
    def __init__(self, max_length=80):
        self.max_length = max_length
        self.current_line_length = 0

    def print(self, text_chunk):
        i = 0
        n = len(text_chunk)
        while i < n:
            start_index = i
            while i < n and text_chunk[i] not in ' \n': # Find end of word
                i += 1
            current_word = text_chunk[start_index:i]

            delimiter = ""
            if i < n: # If not end of chunk, we found a delimiter
                delimiter = text_chunk[i]
                i += 1 # Consume delimiter

            if current_word:
                needs_leading_space = (self.current_line_length > 0)

                # Case 1: Word itself is too long for a line (must be broken)
                if len(current_word) > self.max_length:
                    if needs_leading_space: # Newline if current line has content
                        sys.stdout.write('\n')
                        self.current_line_length = 0
                    for char_val in current_word: # Break the long word
                        if self.current_line_length >= self.max_length:
                            sys.stdout.write('\n')
                            self.current_line_length = 0
                        sys.stdout.write(char_val)
                        self.current_line_length += 1
                # Case 2: Word doesn't fit on current line (print on new line)
                elif self.current_line_length + (1 if needs_leading_space else 0) + len(current_word) > self.max_length:
                    sys.stdout.write('\n')
                    sys.stdout.write(current_word)
                    self.current_line_length = len(current_word)
                # Case 3: Word fits on current line
                else:
                    if needs_leading_space:
                        # Define punctuation that should not have a leading space
                        # when they form an entire "word" (token) following another word.
                        no_leading_space_punctuation = {
                            ",", ".", ";", ":", "!", "?",        # Standard sentence punctuation
                            ")", "]", "}",                     # Closing brackets
                            "'s", "'S", "'re", "'RE", "'ve", "'VE", # Common contractions
                            "'m", "'M", "'ll", "'LL", "'d", "'D",
                            "n't", "N'T",
                            "...", "…"                          # Ellipses
                        }
                        if current_word not in no_leading_space_punctuation:
                            sys.stdout.write(' ')
                            self.current_line_length += 1
                    sys.stdout.write(current_word)
                    self.current_line_length += len(current_word)

            if delimiter == '\n':
                sys.stdout.write('\n')
                self.current_line_length = 0
            elif delimiter == ' ':
                # If line is full and a space delimiter arrives, it implies a wrap.
                if self.current_line_length >= self.max_length:
                    sys.stdout.write('\n')
                    self.current_line_length = 0

        sys.stdout.flush()


In [7]:
# @title Formatted streaming example
from google.colab import ai

wrapper = LineWrapper()
for chunk in ai.generate_text('Give me a long winded description about the evolution of the Roman Empire.', model_name='google/gemini-2.5-flash', stream=True):
  wrapper.print(chunk)

Ah, the Roman Empire! To speak of its evolution is to embark on a sweeping
odyssey through a thousand years of political machination, military conquest,
cultural assimilation, and eventual, agonizing decline. It is not merely a tale
of kings, consuls, and emperors, but a profound narrative of a civilization
constantly reinventing itself, a living organism adapting, growing, fracturing,
and ultimately, transforming into something new.

Our journey begins not with an empire, but a nascent **city-state** on the Tiber
River, shrouded in the mists of myth and legend. From the legendary founding by
Romulus in 753 BCE, through a period of Etruscan dominance, Rome was initially a
**monarchy**. This early phase, though poorly documented, laid crucial
groundwork: the development of a rudimentary civic structure, a militaristic
ethos, and the seeds of a unique legal tradition. But kings, even semi-mythical
ones, eventually became anathema to the burgeoning Roman spirit. The expulsion
of Tarquin t

In [8]:
# @title Formatted streaming example
from google.colab import ai

wrapper = LineWrapper()
for chunk in ai.generate_text('Can you tell me What was the first Roman emperor?', model_name='google/gemini-2.5-flash', stream=True):
  wrapper.print(chunk)

The first Roman Emperor was **Augustus**.

He was originally known as **Gaius Oct avius**, and later as **Octavian**. He
was the adopted son and heir of Julius Caesar. After a period of civil wars
following Caesar's assassination, Octavian emerged victorious.

In **27 BC**, the Roman Senate granted him the honorific title "Augustus,"
marking the generally accepted beginning of the Roman Empire and the end of the
Roman Republic. He ruled for over 40 years, bringing about an era of relative
peace and stability known as the Pax Romana.

In [9]:
from google.colab import ai
wrapper = LineWrapper()
for chunk in ai.generate_text('Are there wars between the Roman Empire and the Persian Empire?',
 model_name='google/gemini-2.5-flash', stream=True
):
    wrapper.print(chunk)

Yes, absolutely! The Roman Empire and various Persian Empires were engaged in
**cent uries of near-constant warfare and rivalry**, arguably the longest and
most impactful geopolitical struggle in ancient and late antique history.

This wasn't just one war, but a series of conflicts that spanned from the 1st
century BCE to the 7th century CE.

Here's a breakdown:

1. **Roman Republic/Early Empire vs. Parthian Empire (c. 53 BCE - 224 CE):**
* **Beginning:** The first major clashes occurred during the late Roman
Republic, most famously with the disastrous defeat of Crassus at the **Battle of
Carrhae in 53 BCE**. This set the tone for Roman-Parthian relations.
* **Ongoing Conflict:** For the next few centuries, Roman emperors like
Augustus, Nero, Trajan, Lucius Verus, Septimius Severus, and Caracalla all
campaigned against the Parthians, often vying for control over buffer states
like Armenia and Mesopotamia. While Rome often achieved tactical victories and
even captured the Parthian capit

In [10]:
from google.colab import ai

stream = ai.generate_text("Tell me What are the reasons for the collapse of the Roman Empire.", stream=True)
for text in stream:
  print(text, end='')

The collapse of the Western Roman Empire in 476 CE (with the deposition of Romulus Augustulus) was not due to a single catastrophic event, but rather a complex interplay of internal weaknesses and external pressures that unfolded over several centuries. Historians debate the precise weight of each factor, but generally agree on the following key reasons:

1.  **Political Instability and Corruption:**
    *   **Frequent Changes in Leadership:** The 3rd Century Crisis (235-284 CE) saw roughly 26 legitimate emperors and numerous usurpers in 50 years, many of whom died violently. This constant upheaval led to civil wars, weakened the central government, and diverted resources.
    *   **Lack of Clear Succession:** There was no established rule for imperial succession, leading to endless power struggles among ambitious generals and politicians.
    *   **Bureaucracy and Corruption:** The vast bureaucracy became increasingly inefficient and corrupt, further draining resources and alienating 

In [11]:
from google.colab import ai
wrapper = LineWrapper(max_length=80)
for chunk in ai.generate_text('Tell me in simple and easy words the reasons for the collapse of the Roman Empire. put in the form of points',

model_name='google/gemini-2.5-flash', stream=True):
    wrapper.print(chunk)

Here are the main reasons why the mighty Roman Empire eventually fell, explained
in simple terms:

* **The Empire Got Too Big:** It became massive, stretching across Europe,
Africa, and Asia, making it incredibly hard to govern, protect, and communicate
across.

* **Money Problems:** The government spent too much on the army and lavish
lifestyles, leading to high taxes, prices going up, and the value of money going
down. Trade suffered.

* **Bad Leaders and Constant Fighting:** Emperors were often murdered, and
different generals fought each other for power. This made the government weak
and unstable, and good decisions were rarely made.

* **Constant Attacks from Outsiders:** Tribes like the Goths and Vandals kept
attacking Roman borders, stretching the army thin and costing a lot of money and
lives.

* ** A Weakening Army:** The Roman army became less effective. They started
hiring foreign soldiers who weren't always loyal to Rome, and couldn't always
defend the vast borders properly

In [12]:
from google.colab import ai
wrapper = LineWrapper(max_length=120)
for chunk in ai.generate_text('Dime en espanoi las razones del colaso del Imperio Romano.',
model_name='google/gemini-2.5-flash', stream=True):
    wrapper.print(chunk)

El colapso del **Imperio Romano de Occidente** (ya que el Imperio Romano de Oriente, conocido como Imperio Bizantino,
perduró casi mil años más) no se debió a una única causa, sino a una compleja interacción de factores económicos,
políticos, militares y sociales que se fueron acumulando a lo largo de varios siglos.

Aquí están las principales razones, agrupadas por categorías:

1. **Problemas Económicos Crónicos:**
* **Inflación y devaluación de la moneda:** Para financiar los enormes gastos militares y burocráticos, los emperadores
empezaron a reducir la cantidad de metales preciosos (oro y plata) en las monedas. Esto llevó a una hiperinflación y a
la desconfianza en el sistema monetario, lo que dificultó el comercio.
* **Altos impuestos:** Para compensar la devaluación y mantener el vasto imperio, se impusieron impuestos cada vez más
pesados a los ciudadanos, especialmente a la clase media y a los agricult ores, lo que sofocó la economía y a menudo
llevó a la rebelión o a la emigrac

In [13]:
from google.colab import ai
wrapper = LineWrapper(max_length=120)
for chunk in ai.generate_text('حدثنى عن اسباب انميار الامبراطورية الرومانية .',
model_name='google/gemini-2.5-flash', stream=True):
    wrapper.print(chunk)

لم يكن انهيار الإمبراطورية الرومانية الغربية (التي سقطت عام 476 م) حدثًا مفاجئًا أو بسبب عامل واحد، بل كان نتيجة لمزيج
معقد من العوامل الداخلية والخارجية التي تفاعلت معًا على مدى قرون. فيما يلي أبرز هذه الأسباب:

1. **عدم الاستقرار السياسي والفساد:**
* **تغيير الأباطرة المتكرر:** خاصة خلال "أزمة القرن الثالث"، حيث تعاقب عشرات الأباطرة في فترة قصيرة، معظمهم قتلوا على يد
جنودهم أو منافسيهم، مما أدى إلى حروب أهل ية مستمرة.
* **صراع القوى:** صراعات داخلية بين الأباطرة والجنرالات والشيوخ أضعفت السلطة المركزية.
* **الفساد الإداري:** الفساد المتفشي في البلاط والإدارة أ دى إلى ضعف الكفاءة ونقص الموارد.

2. **التدهور الاقتصادي:**
* **التضخم:** لتمويل الجيوش الضخمة والحروب، قام الأباطرة بتقليل قيمة العملة المعدنية (بتقليل نسبة الفضة وال ذهب فيها)،
مما أدى إلى تضخم جامح وفقدان الثقة في العملة.
* **ارتفاع الضرائب:** لتعويض نقص الإيرادات وتغطية النفقات العسكرية والإدارية، فُرضت ضرائب باهظة على المواطنين، مما أث قل
كاهل الطبقات المنتجة ودفع الكثيرين إلى الفقر.
* **تراجع التجارة:** عدم الاستقرار السي

## **Reports:**

**We asked Gemini-2.5-flash some questions about frist Roman Emperor used chunck**

**Second question we asked about wars between Roman Empire and Anpersian Empire**

**In the third question we asked about the reasons for the collapse of the Roman Empire**

**In fourth question we asked Gemini-2.5-flash to simple the words  of reasons for the collepse of Roman Empire he gaves simple word and clear reasons easy to understant especially for children or school students**

**We rephrased the question about the reasons for the collapse of the Roman Empire in Spanish and Arabic to see if there is a difference in the context**

**In English context the reasons for the colapse were more comprehensive and in more detail notes we used STREAM**

**In **